<a href="https://colab.research.google.com/github/DenisVasil/Modern_ML/blob/main/KerasNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install --upgrade pip
!pip install -q --upgrade keras-nlp
!pip install -q --upgrade tensorflow
!pip install -q --upgrade keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 36.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [4]:
import os

os.environ["KERAS_BACKEND"] = "jax"

import keras_nlp
import keras

# Mixed precision speeds up training
keras.mixed_precision.set_global_policy("mixed_float16")

In [5]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!# Remove unsupervised examples
!rm -r aclImdb/train/unsup

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  5858k      0  0:00:14  0:00:14 --:--:-- 13.2M


In [6]:
BATCH_SIZE = 16
imdb_train = keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=BATCH_SIZE,
)
imdb_test = keras.utils.text_dataset_from_directory(
    "aclImdb/test",
    batch_size=BATCH_SIZE,
)
print(imdb_train.unbatch().take(1).get_single_element())

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
(<tf.Tensor: shape=(), dtype=string, numpy=b'This is a long lost horror gem starring Sydney Lassick ("Carrie" and others) and Barbara Bach. It is sometimes difficult to locate a copy of this film but it\'s worth it. This film is creepy yet cheesy at the same time. It seems that 3 young newswomen (Karen, Vicky, and Jennifer) travel to the small city of Solvang, California to cover a festival when a mix-up occurs involving their hotel room and they seek refuge at the home of Earnest Keller (Lassick) and his strange wife Virginia. Vickie stays behind, feeling ill, as the other 2 are off to film their story. She is soon murdered at the house, in a VERY cheesy way by some unknown force hiding in the ventilation system (she is decapitated by the closing cover of the vent as it comes crashing down on her while she is being tugged through and into the basement). Soon Karen returns and she is murdered in an even

In [7]:
classifier = keras_nlp.models.BertClassifier.from_preset("bert_tiny_en_uncased_sst2")

100%|██████████| 2.14k/2.14k [00:00<00:00, 3.57MB/s]
100%|██████████| 226k/226k [00:00<00:00, 1.05MB/s]
100%|██████████| 16.8M/16.8M [00:00<00:00, 19.2MB/s]
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'loss_scale_optimizer', because it has 4 variables whereas the saved optimizer has 2 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 0 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [8]:
classifier.predict(["I love modular workflows in keras-nlp!"])

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


array([[-1.539,  1.542]], dtype=float16)

In [9]:
classifier.evaluate(imdb_test)

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1374s 878ms/step - loss: 1.1896 - sparse_categorical_accuracy: 0.5030


[1.185143232345581, 0.5040000081062317]

# Fine-tuning a custom classifier

In [7]:
classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased",
    num_classes=2,
)

100%|██████████| 507/507 [00:00<00:00, 338kB/s]
100%|██████████| 16.8M/16.8M [00:02<00:00, 6.97MB/s]
100%|██████████| 547/547 [00:00<00:00, 547kB/s]
100%|██████████| 226k/226k [00:00<00:00, 300kB/s]


In [8]:
classifier.fit(
    imdb_train,
    validation_data=imdb_test,
    epochs=1,
)

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 54s 31ms/step - loss: 0.6967 - sparse_categorical_accuracy: 0.4912 - val_loss: 0.6934 - val_sparse_categorical_accuracy: 0.5022


In [9]:
classifier.predict(["I love modular workflows in keras-nlp!"])

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


array([[0.1139, 0.1885]], dtype=float16)